<a href="https://colab.research.google.com/github/aru-ai-dev/Hugging-Face-Deep-RL-Course/blob/main/Deep_RL_Agent_Unit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install swig cmake

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [ ]:
# Install virtual screen libraries and create and run a virtual screen
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# This code crashes the runtime
import os
os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [ ]:
import gymnasium as gym

# Create an env called LunarLander-v2
env = gym.make("LunarLander-v2")

# Reset the env
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken: ", action)

  # Do this action and get next_state, reward, terimnated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If landed or crashed
  if terminated or truncated:
    print("Env is reset")
    observation, info = env.reset()

env.close()

Action taken:  1
Action taken:  3
Action taken:  2
Action taken:  0
Action taken:  2
Action taken:  3
Action taken:  0
Action taken:  2
Action taken:  3
Action taken:  1
Action taken:  0
Action taken:  3
Action taken:  3
Action taken:  3
Action taken:  3
Action taken:  3
Action taken:  2
Action taken:  1
Action taken:  1
Action taken:  0


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/

In [ ]:
env = gym.make("LunarLander-v2")
env.reset
print("OBSERVATION SPACE")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())


# Observation is a vector of size 8
# info about coordinates, speed etc

OBSERVATION SPACE
Observation Space Shape (8,)
Sample observation [57.64828    53.870903   -1.3687514   0.08277864  0.9166876   0.48421016
  0.31677732  0.20648052]


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print("ACTION SPACE")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())

# Action space is discrete 4 actions

ACTION SPACE
Action Space Shape 4
Action Space Sample 1


In [ ]:
# Vectorized env is a method of stacking multiple independent envs to a single env
env = make_vec_env("LunarLander-v2", n_envs=16)

In [ ]:
# Defining the PPO architecture
# Using MuliLayerPerceptronPolicy because input is a vector
# We would use CnnPolicy if the input were frames
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose = 1)

Using cuda device


In [ ]:
# Training the PPO Agent for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
model_name = "ppo-LunarLander-v2"
model.save(model_name)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.6     |
|    ep_rew_mean     | -178     |
| time/              |          |
|    fps             | 3338     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 89.6        |
|    ep_rew_mean          | -133        |
| time/                   |             |
|    fps                  | 2290        |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.006380397 |
|    clip_fraction        | 0.0379      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | -0.00147    |
|    learning_rate        | 0.

In [ ]:
# Evaluating the agent
eval_env = Monitor(gym.make("LunarLander-v2", render_mode= 'rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=271.72 +/- 21.4067802811709


In [ ]:
notebook_login()
!git config --global credential.helper store

In [ ]:
# Push the trained agent
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

env_id = "LunarLander-v2"

model_architecture = "PPO"

repo_id = "devaru-ai/ppo-LunarLander-v2"
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

package_to_hub(model=model, # The trained model
               model_name=model_name, # Name of the trained model
               model_architecture=model_architecture, # The model architecture used
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id,
               commit_message=commit_message)

In [ ]:
# Since the model downloaded from the Hub was trained with Gym  we need to install shimmy a API conversion tool that will help us to run the environment correctly
!pip install shimmy

In [9]:
from huggingface_sb3 import load_from_hub
repo_id = "devaru-ai/ppo-LunarLander-v2" # The repo_id
filename = "ppo-LunarLander-v2.zip" # The model filename.zip

custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
}

checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, custom_objects=custom_objects, print_system_info=True)

ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

== CURRENT SYSTEM INFO ==
- OS: Linux-6.1.85+-x86_64-with-glibc2.35 # 1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
- Python: 3.11.11
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.5.1+cu121
- GPU Enabled: True
- Numpy: 1.26.4
- Cloudpickle: 3.1.0
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-6.1.85+-x86_64-with-glibc2.35 # 1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
- Python: 3.11.11
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.5.1+cu121
- GPU Enabled: True
- Numpy: 1.26.4
- Cloudpickle: 3.1.0
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2



/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_content, map_locati

In [10]:
# Evaluating the agent
import gymnasium as gym
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=254.50 +/- 25.064183046625182
